In [1]:
!pip install "audio-separator[gpu]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.8/917.8 kB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.8/383.8 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from audio_separator.separator import Separator

separator = Separator()

# Example: load Demucs
separator.load_model('htdemucs_6s.yaml')

# Or try MDX Extra
# separator.load_model("mdx_extra")

output_files = separator.separate("/content/audio_example.mp3")
print(f"Separation complete! Output file(s): {' '.join(output_files)}")


INFO:audio_separator.separator.separator:Separator version 0.39.1 instantiating with output_dir: None, output_format: WAV
INFO:audio_separator.separator.separator:Output directory not specified. Using current working directory.
INFO:audio_separator.separator.separator:Using model directory from model_file_dir parameter: /tmp/audio-separator-models/
INFO:audio_separator.separator.separator:Operating System: Linux #1 SMP Thu Oct  2 10:42:05 UTC 2025
INFO:audio_separator.separator.separator:System: Linux Node: d32c59a071b7 Release: 6.6.105+ Machine: x86_64 Proc: x86_64
INFO:audio_separator.separator.separator:Python Version: 3.12.12
INFO:audio_separator.separator.separator:PyTorch Version: 2.8.0+cu126
INFO:audio_separator.separator.separator:FFmpeg installed: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
INFO:audio_separator.separator.separator:ONNX Runtime GPU package installed with version: 1.23.2
INFO:audio_separator.separator.separator:CUDA is ava

Separation complete! Output file(s): audio_example_(Bass)_htdemucs_6s.wav audio_example_(Drums)_htdemucs_6s.wav audio_example_(Other)_htdemucs_6s.wav audio_example_(Vocals)_htdemucs_6s.wav audio_example_(Guitar)_htdemucs_6s.wav audio_example_(Piano)_htdemucs_6s.wav


In [3]:
%ls

'audio_example_(Bass)_htdemucs_6s.wav'
'audio_example_(Drums)_htdemucs_6s.wav'
'audio_example_(Guitar)_htdemucs_6s.wav'
 audio_example.mp3
'audio_example_(Other)_htdemucs_6s.wav'
'audio_example_(Piano)_htdemucs_6s.wav'
'audio_example_(Vocals)_htdemucs_6s.wav'
 sample_data/


In [4]:
!pip install pyngrok

In [5]:
!pip install nest_asyncio


In [6]:
!ngrok config add-authtoken 30eW25DS9DAjbcnfLl0dgJGLi26_3cRvbDMAmAjCXVeyi6pmt

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
import shutil
import uuid
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import FileResponse
from fastapi.staticfiles import StaticFiles
from audio_separator.separator import Separator
from pyngrok import ngrok

app = FastAPI(title="Audio Separation API")
os.makedirs("user_data", exist_ok=True)
# Mount static folder for serving separated files
app.mount("/static", StaticFiles(directory="user_data"), name="static")

# Load model once globally
separator = Separator()
separator.load_model("htdemucs_6s.yaml")

def get_rename_mapping(base_name: str):
    return {
        f"{base_name}_(Bass)_htdemucs_6s.wav": "bass.wav",
        f"{base_name}_(Piano)_htdemucs_6s.wav": "piano.wav",
        f"{base_name}_(Other)_htdemucs_6s.wav": "strings_or_pads.wav",
        f"{base_name}_(Vocals)_htdemucs_6s.wav": "vocals.wav",
        f"{base_name}_(Guitar)_htdemucs_6s.wav": "guitar.wav",
        f"{base_name}_(Drums)_htdemucs_6s.wav": "drums.wav",
    }

@app.post("/separate/")
async def separate_audio(file: UploadFile = File(...)):
    user_id = str(uuid.uuid4())
    user_folder = os.path.join("user_data", user_id)
    os.makedirs(user_folder, exist_ok=True)

    input_path = os.path.join(user_folder, file.filename)
    with open(input_path, "wb") as f:
        shutil.copyfileobj(file.file, f)

    output_files = separator.separate(input_path)

    base_name, _ = os.path.splitext(file.filename)
    rename_mapping = get_rename_mapping(base_name)

    renamed_files = []
    for old_path in output_files:
        old_name = os.path.basename(old_path)
        if old_name in rename_mapping:
            new_path = os.path.join(user_folder, rename_mapping[old_name])
            os.rename(old_path, new_path)
            renamed_files.append(new_path)

    file_urls = [
        f"/static/{user_id}/{os.path.basename(f)}" for f in renamed_files
    ]

    return {"user_id": user_id, "files": file_urls}

@app.get("/download/{user_id}/{filename}")
async def download_file(user_id: str, filename: str):
    file_path = os.path.join("user_data", user_id, filename)
    if os.path.exists(file_path):
        return FileResponse(file_path, filename=filename)
    return {"error": "File not found"}

if __name__ == "__main__":
    import uvicorn
    from pyngrok import ngrok

    # 🔑 Set your ngrok token here
    NGROK_TOKEN = "30eW25DS9DAjbcnfLl0dgJGLi26_3cRvbDMAmAjCXVeyi6pmt"
    ngrok.set_auth_token(NGROK_TOKEN)

    # Start a tunnel on port 8000
    public_url = ngrok.connect(8000)
    print(f"Public URL: {public_url}")

    # Run FastAPI app asynchronously in the existing event loop
    config = uvicorn.Config(app, host="0.0.0.0", port=8000)
    server = uvicorn.Server(config)
    await server.serve()

INFO:audio_separator.separator.separator:Separator version 0.39.1 instantiating with output_dir: None, output_format: WAV
INFO:audio_separator.separator.separator:Output directory not specified. Using current working directory.
INFO:audio_separator.separator.separator:Using model directory from model_file_dir parameter: /tmp/audio-separator-models/
INFO:audio_separator.separator.separator:Operating System: Linux #1 SMP Thu Oct  2 10:42:05 UTC 2025
INFO:audio_separator.separator.separator:System: Linux Node: d32c59a071b7 Release: 6.6.105+ Machine: x86_64 Proc: x86_64
INFO:audio_separator.separator.separator:Python Version: 3.12.12
INFO:audio_separator.separator.separator:PyTorch Version: 2.8.0+cu126
INFO:audio_separator.separator.separator:FFmpeg installed: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
INFO:audio_separator.separator.separator:ONNX Runtime GPU package installed with version: 1.23.2
INFO:audio_separator.separator.separator:CUDA is ava

Public URL: NgrokTunnel: "https://2ce6d2482aa9.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [2065]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     103.189.158.29:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     103.189.158.29:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     103.189.158.29:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     103.189.158.29:0 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:audio_separator.separator.separator:Processing file: user_data/d88418b0-58a9-4948-8157-e11092941001/audio_example.mp3
INFO:audio_separator.separator.separator:Starting separation process for audio_file_path: user_data/d88418b0-58a9-4948-8157-e11092941001/audio_example.mp3
100%|██████████| 2/2 [00:00<00:00,  4.92it/s]
INFO:audio_separator.separator.separator:Audio duration is 0.00 hours (10.88 seconds).
INFO:audio_separator.separator.separator:Using pydub for writing.
INFO:audio_separator.separator.separator:Audio duration is 0.00 hours (10.88 seconds).
INFO:audio_separator.separator.separator:Using pydub for writing.
INFO:audio_separator.separator.separator:Audio duration is 0.00 hours (10.88 seconds).
INFO:audio_separator.separator.separator:Using pydub for writing.
INFO:audio_separator.separator.separator:Audio duration is 0.00 hours (10.88 seconds).
INFO:audio_separator.separator.separator:Using pydub for writing.
INFO:audio_separator.separator.separator:Audio duration is 0.00 

INFO:     103.189.158.29:0 - "POST /separate/ HTTP/1.1" 200 OK
INFO:     103.189.158.29:0 - "GET /download/d88418b0-58a9-4948-8157-e11092941001/guitar.wav HTTP/1.1" 200 OK


In [ ]:
!audio-separator --list_models